### Import Libraries

In [1]:
import pickle
import os
import pandas as pd
import enum
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from spear.labeling import PreLabels
import numpy as np
from spear.cage import Cage
from utils import custom_dataset, train_all_LF
from generate_LF import get_variables
from spear.labeling import labeling_function, ABSTAIN, preprocessor, continuous_scorer
import re


In [2]:
from labelling_function import *
from PIL import Image
from resnet18 import ResNet, BasicBlock
import torch
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import torchvision.models as models
from resnet18 import ResNet, BasicBlock
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import cv2

### Declaring ClassLabels

In [3]:
ABSTAIN = None

class ClassLabels(enum.Enum):
    LYMPHOCYTE= 0
    NONLYMPHOCYTE = 1
    
    
   

### Defining Labelling Functions

#### LFs for Class 0

In [4]:
from spear.labeling import labeling_function, ABSTAIN, preprocessor, continuous_scorer
import re
# path = "/home/akshit/Desktop/MICCAI/data/models/"
path = "/home/raja/Desktop/MICCAI/data/models/100/"

# SVM
@continuous_scorer()
def svm_V1(x,**kwargs):
    import pickle
    import numpy as np

    x = np.array(x).flatten() # x is 28x28x3 input img 
    svm = pickle.load(open(path+'0_svmV11.pkl','rb'))
    confidence_scores = svm.predict_proba([x])
    # print(confidence_scores)
    return float(confidence_scores[0][0]) #only called when model doesn't ABSTAIN, output confidence on class label

@labeling_function(cont_scorer=svm_V1, label=ClassLabels.LYMPHOCYTE)
def LF_svm0_V1(x, **kwargs):
    import pickle
    import numpy as np

    x = np.array(x).flatten()  
    svm = pickle.load(open(path+'0_svmV11.pkl','rb'))
    
    if svm.predict_proba([x])[0][0]>0.50: 
        return ClassLabels.LYMPHOCYTE # Return label only if confidence > 0.8
    else: 
        return ABSTAIN
    



In [5]:
import torch
import pickle
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_resnet_model(model_path):
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    return model

def load_image_with_transform(input_img):
    input_img = input_img.copy()
    input_img = Image.fromarray(input_img)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

    input_img = transform(input_img)
    input_img = input_img.clone().detach().requires_grad_(True).to(torch.float)

    #input_img = torch.tensor(input_img.clone().detach().requires_grad_(True), dtype=torch.float)
    input_img = input_img.unsqueeze(0).to(device)

    return input_img

model_path = '/home/raja/Desktop/MICCAI/data/models/100/best.pkl'
model = load_resnet_model(model_path)

@labeling_function(label=ClassLabels.NONLYMPHOCYTE)
def ResNet18LF_NONLYMPHOCYTE(x,**kwargs):
    input_image = load_image_with_transform(x)
    output = model(input_image)
    predicted_class = torch.argmax(output, dim=1)
    confidence_scores = torch.nn.functional.softmax(output, dim=1)

    if predicted_class.item() == 1 and confidence_scores[0][1] > 0.65:
        return ClassLabels.NONLYMPHOCYTE
    else:
        return ABSTAIN


In [6]:
import torch
import pickle
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_resnet_model(model_path):
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    return model

def load_image_with_transform(input_img):
    input_img = input_img.copy()
    input_img = Image.fromarray(input_img)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
#sourceTensor.clone().detach().requires_grad_(True)
    input_img = transform(input_img)
    input_img = input_img.clone().detach().requires_grad_(True).to(torch.float)

    #input_img = torch.tensor(input_img.clone().detach().requires_grad_(True), dtype=torch.float)
    input_img = input_img.unsqueeze(0).to(device)

    return input_img

model_path = '/home/raja/Desktop/MICCAI/data/models/100/best.pkl'
model = load_resnet_model(model_path)

@labeling_function(label=ClassLabels.LYMPHOCYTE)
def ResNet18LF_LYMPHOCYTE(x,**kwargs):
    input_image = load_image_with_transform(x)
    output = model(input_image)
    predicted_class = torch.argmax(output, dim=1)
    confidence_scores = torch.nn.functional.softmax(output, dim=1)

    if predicted_class.item() == 0 and confidence_scores[0][0] > 0.50:
        return ClassLabels.LYMPHOCYTE
    else:
        return ABSTAIN


#### LFs for Class 1

In [7]:
# SVM
@continuous_scorer()
def svm1_V1(x,**kwargs):
    import pickle
    import numpy as np

    x = np.array(x).flatten() # x is 28x28x3 input img 
    svm = pickle.load(open(path+'1_svmV11.pkl','rb'))
    confidence_scores = svm.predict_proba([x])

    return float(confidence_scores[0][1]) #only called when model doesn't ABSTAIN, output confidence on class label

@labeling_function(cont_scorer=svm1_V1, label=ClassLabels.NONLYMPHOCYTE)
def LF_svm1_V1(x, **kwargs):
    import pickle
    import numpy as np

    x = np.array(x).flatten()  
    svm = pickle.load(open(path+'1_svmV11.pkl','rb'))
    
    if svm.predict_proba([x])[0][1]>0.6: 
        return ClassLabels.NONLYMPHOCYTE
    else: 
        return ABSTAIN


In [8]:
from labelling_function import *

@labeling_function(label=ClassLabels.NONLYMPHOCYTE)
def LFNL1(c,**kwargs): 
    red_score = calculate_brown_score(c)
    blue_score = calculate_blue_score(c)
    green_score = calculate_green_score(c)
    if blue_score > 155:
        return ClassLabels.NONLYMPHOCYTE
    else:
        return ABSTAIN
    
@labeling_function(label=ClassLabels.NONLYMPHOCYTE)
def LFNL2(c,**kwargs): 
    red_score = calculate_brown_score(c)
    blue_score = calculate_blue_score(c)
    if 10 < blue_score < 180 and red_score > 170:
        return ClassLabels.NONLYMPHOCYTE
    else:
        return ABSTAIN    
    
@labeling_function(label=ClassLabels.NONLYMPHOCYTE)
def LFNL9(c,**kwargs): 
    green_score = calculate_green_score(c)
    blue_score = calculate_blue_score(c)
    if 10 < blue_score < 180 and green_score > 155:
        return ClassLabels.NONLYMPHOCYTE
    else:
        return ABSTAIN      
    
@labeling_function(label=ClassLabels.NONLYMPHOCYTE)
def LFNL3(c,**kwargs): 
    brown_score = compute_brown_score(c)
    total_blue_score = compute_blue_score(c)
    if total_blue_score < 12:
        return ClassLabels.NONLYMPHOCYTE
    else:
        return ABSTAIN   


@labeling_function(label=ClassLabels.NONLYMPHOCYTE)
def LFNL4(c,**kwargs): 
    brown_score = compute_brown_score(c)
    total_blue_score = compute_blue_score(c)
    if brown_score > 585:
        return ClassLabels.NONLYMPHOCYTE
    else:
        return ABSTAIN   

@labeling_function(label=ClassLabels.NONLYMPHOCYTE)
def LFNL5(c,**kwargs): 
    red_score = calculate_brown_score(c)
    blue_score = calculate_blue_score(c)
    green_score = calculate_green_score(c)
    if red_score > 170:
        return ClassLabels.NONLYMPHOCYTE
    else:
        return ABSTAIN


@labeling_function(label=ClassLabels.NONLYMPHOCYTE)
def LFNL6(c,**kwargs): 
    red_score = calculate_brown_score(c)
    blue_score = calculate_blue_score(c)
    green_score = calculate_green_score(c)
    if red_score < 12:
        return ClassLabels.NONLYMPHOCYTE
    else:
        return ABSTAIN

@labeling_function(label=ClassLabels.NONLYMPHOCYTE)
def LFNL7(c,**kwargs): 
    red_score = calculate_brown_score(c)
    blue_score = calculate_blue_score(c)
    green_score = calculate_green_score(c)
    if green_score > 155:
        return ClassLabels.NONLYMPHOCYTE
    else:
        return ABSTAIN

@labeling_function(label=ClassLabels.NONLYMPHOCYTE)
def LFNL8(c,**kwargs): 
    black_score = compute_black_score(c)
    if black_score > 9:
        return ClassLabels.NONLYMPHOCYTE
    else:
        return ABSTAIN           

    
                 

## Aggregating LFs & Labelling dataset 

### LFSet
Placeholder for declared LFs

In [9]:
from spear.labeling import LFSet

LFS = [
    #ResNet18LF_LYMPHOCYTE,
    #ResNet18LF_NONLYMPHOCYTE,
    LFNL1,
    LFNL2,
    LFNL3,
    LFNL4,
    LFNL5,
    LFNL6,
    LFNL7,
    LFNL8,
    LFNL9,
    
    LF_svm0_V1,
    LF_svm1_V1,  
  
]

QT2 = 0.99
QC2 = 0.99

qt1 = np.array([0.9999,0.9999,0.9999,0.9999,0.9999,0.9999,0.9999,0.9999,0.9999])
qc1 = np.array([0.99999,0.99999,0.9999,0.9999,0.9999,0.9999,0.9999,0.9999,0.9999])

rules = LFSet("BM_LF")
rules.add_lf_list(LFS)

### Load the dataset

In [10]:
from utils import custom_dataset, train_all_LF
from generate_LF import get_variables
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

classes,label_frac,data_path,save_path = get_variables()
dataset,x,y = custom_dataset(classes=classes,path=data_path ,fraction=label_frac)



2023-05-24 17:08:18.856745: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-24 17:08:18.932589: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-24 17:08:19.257486: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/raja/Desktop/cage/lib/python3.10/site-packages/cv2/../../lib64:
2023-05-24 17:08:19.257524: 

## Cage

In [11]:
def cage_loop(LFS, max_iters=10, threshold=10**-5, img_per_class = 150):
    from keras.utils import np_utils
    # Paths
    log_path_cage = './cage_loop/log.txt' 
    params_path = None
    path_json = "./cage_loop/labels.json"
    U_path_pkl = "./cage_loop/unlabelled.pkl"
    L_path_pkl = "./cage_loop/labelled.pkl"

    # Loading Data
    classes,label_frac,data_path,save_path = get_variables()
    print("Classes used in expt:",classes)
    dataset,x,y = custom_dataset(classes=classes, path=data_path, fraction=label_frac)

    xu = np.array(dataset['test_images'])
    yu = np.array(dataset['test_labels'])
    print(np.shape(xu),np.shape(yu))
    # Creating rules
    n_lfs = len(LFS)
    rules = LFSet("BM_LF")
    rules.add_lf_list(LFS)
    
    confidence_list = []
    val_scores = []

    for i in range(max_iters):
        # Train Models in LFs
        train_all_LF(x,y,len(classes),save_path,label_frac)

        # Unlabelled
        u_noisy_labels = PreLabels(name="bmnist_rem_ul",
                                    data=xu,
                                    rules=rules,
                                    labels_enum=ClassLabels,
                                    num_classes=len(classes))
        # Lu,Su = u_noisy_labels.get_labels()
        u_noisy_labels.generate_pickle(U_path_pkl)

        # Labelled
        l_noisy_labels = PreLabels(name="bmnist_l",
                                    data=x,
                                    gold_labels=y,
                                    rules=rules,
                                    labels_enum=ClassLabels,
                                    num_classes=len(classes))
        # Ll,Sl = l_noisy_labels.get_labels()
        l_noisy_labels.generate_pickle(L_path_pkl)
        l_noisy_labels.generate_json(path_json)


        # Cage
        cage = Cage(path_json = path_json, n_lfs = n_lfs)
        if params_path is not None: 
            cage.load_params(load_path = params_path)
        else:
            params_path = './cage_loop/params.pkl' 
        
        probs = cage.fit_and_predict_proba(path_pkl = U_path_pkl, path_test = L_path_pkl, path_log = log_path_cage, qt = QT2, qc = QC2, metric_avg = ['macro'], n_epochs = 100, lr = 0.01)
        labels = np.argmax(probs, 1)

        print(labels)

        values, frequency = np.unique(yu, return_counts=True)
        for values, frequency in zip(values, frequency):
            print(f"Labels of Lake Class {values}: {frequency}")

        values, frequency = np.unique(y, return_counts=True)
        for values, frequency in zip(values, frequency):
            print(f"Labels of Labelled Set {values}: {frequency}")
        
        print("="*45)
        print("Iteration",i)
        print("Shape of Labeled Data:",x.shape)
        print("Shape of Unlabeled Data:",xu.shape)
        print("Accuracy on unlabelled images:",accuracy_score(labels,yu)*100)
        print("="*45)
        
        cage.save_params(save_path = params_path)

        confidence = np.array([np.max(i) for i in probs])
        confidence_list.append(confidence)
        print(i,probs.shape)

        # Getting indices of probabilities in decreasing order
        idx = np.argsort(confidence)
        idx = idx[::-1] 

        # Number of images per class (5%)
        # img_per_class = int(0.05*len(confidence)/len(classes))

        # Number of images per class (50)
        
        
        print("Num img per class =",img_per_class)

        pop_list = [] #list of indices of images to be added
        label_count = []

        for j in idx:
            if confidence[j]>threshold and label_count.count(labels[j])<img_per_class:
                pop_list.append(j)
                label_count.append(labels[j])
        
        print("Number of images getting transferred:", len(pop_list))
        print('Accuracy of Pseudo-labelled img added to dataset:', accuracy_score(labels[pop_list],yu[pop_list])*100)
        
        x = np.append(x,xu[pop_list], axis=0)
        y = np.append(y,labels[pop_list], axis=0)        
        xu = np.delete(xu,pop_list, axis=0)
        yu = np.delete(yu,pop_list, axis=0)

        if len(pop_list)<5:
            break

        # Deleting variables
        del u_noisy_labels
        del l_noisy_labels
        del cage

        classes,label_frac,data_path,save_path = get_variables()


    return x,y,xu,yu,confidence_list


In [12]:
#  img_per_class: num images added per loop per class
x,y,xu,yu,confidence_list = cage_loop(LFS, max_iters=1, threshold=10**-10,  img_per_class = 10)

Classes used in expt: [0, 1]
(2, 30, 30, 3) (2,)


/home/raja/Desktop/cage/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Trained & Saved 6 models


/home/raja/Desktop/cage/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Trained & Saved 6 models


100%|██████████| 100/100 [00:02<00:00, 39.54it/s]

final_test_accuracy_score: 0.9523809523809523
test_average_metric: macro	final_test_f1_score: 0.9523420479302832
[0 0]
Labels of Lake Class 0: 1
Labels of Lake Class 1: 1
Labels of Labelled Set 0: 105
Labels of Labelled Set 1: 105
Iteration 0
Shape of Labeled Data: (210, 30, 30, 3)
Shape of Unlabeled Data: (2, 30, 30, 3)
Accuracy on unlabelled images: 50.0
0 (2, 2)
Num img per class = 10
Number of images getting transferred: 2
Accuracy of Pseudo-labelled img added to dataset: 50.0


In [13]:
confidence_list

[array([0.00109503, 0.00109503])]

In [17]:
import os
import imageio
import numpy as np
from skimage.transform import resize

image_folder = '/home/raja/Desktop/MICCAI/training_folder'

#image_folder = '/home/raja/Desktop/segment-anything/output/NONLYM'
#image_folder = '/home/raja/Desktop/segment-anything/output/TRAIN/LYMPHOCYTES'
image_files = os.listdir(image_folder)

image_data = []
image_names = []
for file in image_files:
    image_path = os.path.join(image_folder, file)
    image = imageio.imread(image_path)
    resized_image = resize(image, (30, 30, 3))
    image_data.append(resized_image)
    image_names.append(file)

xuu = (np.array(image_data) * 255).astype(np.uint8)

print(xuu.shape)  # (num_images, 30, 30, 3)
#print(image_names)  # list of image file names in the same order as x


/tmp/ipykernel_31031/3000661952.py:16: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(image_path)


(370437, 30, 30, 3)


In [22]:

from keras.utils import np_utils
# Paths
log_path_cage = './cage_loop/log.txt' 
params_path = None
path_json = "./cage_loop/labels.json"
U_path_pkl = "./cage_loop/unlabelled.pkl"
L_path_pkl = "./cage_loop/labelled.pkl"

# Loading Data
classes,label_frac,data_path,save_path = get_variables()
#print("Classes used in expt:",classes)
dataset,x,y = custom_dataset(classes=classes, path=data_path, fraction=label_frac)
xu = np.array(dataset['test_images'])

# Unlabelled
u_noisy_labels = PreLabels(name="bmnist_rem_ul",
                            data=xuu,
                            rules=rules,
                            labels_enum=ClassLabels,
                            num_classes=len(classes))
# Lu,Su = u_noisy_labels.get_labels()
u_noisy_labels.generate_pickle(U_path_pkl)

# Labelled
l_noisy_labels = PreLabels(name="bmnist_l",
                            data=x,
                            gold_labels=y,
                            rules=rules,
                            labels_enum=ClassLabels,
                            num_classes=len(classes))
# Ll,Sl = l_noisy_labels.get_labels()
l_noisy_labels.generate_pickle(L_path_pkl)
l_noisy_labels.generate_json(path_json)




#print(np.shape(xuu))
# Creating rules
n_lfs = len(LFS)
rules = LFSet("BM_LF")
rules.add_lf_list(LFS)

# Cage
cage = Cage(path_json = path_json, n_lfs = n_lfs)
if params_path is not None: 
    cage.load_params(load_path = params_path)
else:
    params_path = './cage_loop/params.pkl' 
probs = cage.fit_and_predict_proba(path_pkl = U_path_pkl, path_test = L_path_pkl, path_log = log_path_cage, qt = QT2, qc = QC2, metric_avg = ['macro'], n_epochs = 100, lr = 0.01)

#probs = cage.fit_and_predict_proba(path_pkl = U_path_pkl, path_test = L_path_pkl, path_log = log_path_cage, qt = 0.9, qc = 0.85, metric_avg = ['macro'], n_epochs = 100, lr = 0.01)
labels = np.argmax(probs, 1)

print(labels)

# Define the file path to save the probabilities
probs_file_path = '/home/raja/Desktop/MICCAI/probs.txt'

# Save the probabilities to the text file
np.savetxt(probs_file_path, probs)






100%|██████████| 100/100 [00:03<00:00, 26.08it/s]


final_test_accuracy_score: 0.9476190476190476
test_average_metric: macro	final_test_f1_score: 0.947560781821071
[1 1 1 ... 1 1 1]


In [24]:
import os
import imageio
import numpy as np
from skimage.transform import resize

image_folder = '/home/raja/Desktop/MICCAI/test_folder'

#image_folder = '/home/raja/Desktop/segment-anything/output/NONLYM'
#image_folder = '/home/raja/Desktop/segment-anything/output/TRAIN/LYMPHOCYTES'
image_files = os.listdir(image_folder)

image_data = []
image_names = []
for file in image_files:
    image_path = os.path.join(image_folder, file)
    image = imageio.imread(image_path)
    resized_image = resize(image, (30, 30, 3))
    image_data.append(resized_image)
    image_names.append(file)

xuu = (np.array(image_data) * 255).astype(np.uint8)

print(xuu.shape)  # (num_images, 30, 30, 3)
#print(image_names)  # list of image file names in the same order as x


/tmp/ipykernel_31031/1530542732.py:16: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(image_path)


(46376, 30, 30, 3)


In [35]:
from keras.utils import np_utils
# Paths
log_path_cage = './cage_loop/log.txt' 
params_path = None
path_json = "./cage_loop/labels.json"
U_path_pkl = "./cage_loop/unlabelled.pkl"
L_path_pkl = "./cage_loop/labelled.pkl"

# Loading Data
classes,label_frac,data_path,save_path = get_variables()
#print("Classes used in expt:",classes)
dataset,x,y = custom_dataset(classes=classes, path=data_path, fraction=label_frac)
xu = np.array(dataset['test_images'])

# Unlabelled
u_noisy_labels = PreLabels(name="bmnist_rem_ul",
                            data=xuu,
                            rules=rules,
                            labels_enum=ClassLabels,
                            num_classes=len(classes))
# Lu,Su = u_noisy_labels.get_labels()
u_noisy_labels.generate_pickle(U_path_pkl)

# Labelled
l_noisy_labels = PreLabels(name="bmnist_l",
                            data=x,
                            gold_labels=y,
                            rules=rules,
                            labels_enum=ClassLabels,
                            num_classes=len(classes))
# Ll,Sl = l_noisy_labels.get_labels()
l_noisy_labels.generate_pickle(L_path_pkl)
l_noisy_labels.generate_json(path_json)




#print(np.shape(xuu))
# Creating rules
n_lfs = len(LFS)
rules = LFSet("BM_LF")
rules.add_lf_list(LFS)

# Cage
cage = Cage(path_json = path_json, n_lfs = n_lfs)
if params_path is not None: 
    cage.load_params(load_path = params_path)
else:
    params_path = './cage_loop/params.pkl' 
probs = cage.predict_proba(path_test = U_path_pkl, qc = QC2)    
labels = np.argmax(probs, 1)

100%|██████████| 210/210 [00:00<00:00, 605.02it/s]


In [36]:
len(labels)

46376

In [37]:
labels

array([1, 1, 1, ..., 1, 1, 0])

In [38]:
import csv

# Combine image names and predicted labels
data = zip(image_names, labels)

# Write to CSV file
with open('/home/raja/Desktop/MICCAI/predictions-train-SVM-HF-cage-prob.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Image Name', 'Label'])
    for row in data:
        writer.writerow(row)

In [40]:
import csv
import re

# Define input and output file names
input_file = '/home/raja/Desktop/MICCAI/predictions-train-SVM-HF-cage-prob.csv'
output_file = '/home/raja/Desktop/MICCAI/output-train-final-SVM-HF-prob.csv'

# Define dictionary to store counts for each image
image_counts = {}

# Define regular expression to extract image number from filename
img_regex = re.compile(r'img_(\d+)_cropped_\d+.jpg')

# Read input file and count labels for each image
with open(input_file, 'r') as f:
    reader = csv.reader(f)
    next(reader)  # skip header row
    for row in reader:
        image_name, label = row
        img_match = img_regex.match(image_name)
        if img_match:
            image_number = int(img_match.group(1))
            if image_number in image_counts:
                if label == '0':
                    image_counts[image_number][0] += 1
                else:
                    image_counts[image_number][1] += 1
            else:
                if label == '0':
                    image_counts[image_number] = {0: 1, 1: 0}
                else:
                    image_counts[image_number] = {0: 0, 1: 1}

# Write output file with counts for each image
with open(output_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Image', 'Predicted-Lymphocyte', 'Non-Lymphocyte'])
    for image_number, counts in sorted(image_counts.items()):
        writer.writerow(['img_{}'.format(image_number), counts[0], counts[1]])


In [ ]:
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('/home/raja/Desktop/MICCAI/output-train-final-SVM-HF-prob.csv')

# Create a new DataFrame with the full range of image numbers (18000 to 19999)
img_range = range(18000, 20000)
new_df = pd.DataFrame({'Image': ['img_' + str(i) for i in img_range],
                       'Predicted-Lymphocyte': 0,
                       'Non-Lymphocyte': 0})

# Merge the new DataFrame with the original DataFrame using a left join
merged_df = pd.merge(new_df, df, on='Image', how='left')

# Fill the NaN values with zeros
merged_df.fillna(0, inplace=True)

# Write the merged DataFrame to a new CSV file
merged_df.to_csv('/home/raja/Desktop/MICCAI/new_file-itr2-prob.csv', index=False)


In [41]:
import pandas as pd

# read the csv file into a pandas DataFrame
df = pd.read_csv('/home/raja/Desktop/MICCAI/output-train-final-SVM-HF-prob.csv')

# create a new DataFrame with the full range of image numbers
img_range = range(2000)
new_df = pd.DataFrame({'Image': ['img_' + str(i) for i in img_range], 'Predicted-Lymphocyte': 0})

# merge the new DataFrame with the original DataFrame using a left join
merged_df = pd.merge(new_df, df, on='Image', how='left')


# write the merged DataFrame to a new csv file
merged_df.to_csv('/home/raja/Desktop/MICCAI/new_file-itr2-prob.csv', index=False)

# Load the Excel file
df = pd.read_csv('/home/raja/Desktop/MICCAI/new_file-itr2-prob.csv')

# Fill the 'predicted' column with 0 for NA values
df['Predicted-Lymphocyte_y'].fillna(0, inplace=True)

# Save the modified DataFrame back to Excel
df.to_csv('/home/raja/Desktop/MICCAI/new_file-itr2-prob.csv', index=False)


In [ ]:
'''
import pandas as pd

# read the input CSV file into a pandas dataframe
df = pd.read_csv('/home/raja/Desktop/MICCAI/new_file-SVM-HF.csv')

# define the function to assign class labels based on ground truth values
def assign_class(gt):
    if gt == 0:
        return '0 (No lymphocytes)'
    elif 1 <= gt <= 5:
        return '1-5'
    elif 6 <= gt <= 10:
        return '6-10'
    elif 11 <= gt <= 20:
        return '11-20'
    elif 21 <= gt <= 50:
        return '21-50'
    elif 51 <= gt <= 200:
        return '51-200'
    else:
        return '>200'

# assign class labels to ground truth values
df['Ground-truth Class'] = df['Ground-truth'].apply(assign_class)

# assign class labels to predicted values based on the constraints
df['Predicted Class'] = df.apply(lambda row: assign_class(row['Predicted-Lymphocyte_y']), axis=1)

# calculate the confusion matrix
confusion_matrix = pd.crosstab(df['Ground-truth Class'], df['Predicted Class'])

# define the desired order of the classes
desired_order = ['0 (No lymphocytes)', '1-5', '6-10', '11-20', '21-50']

# reindex the confusion matrix to match the desired order
confusion_matrix = confusion_matrix.reindex(index=desired_order, columns=desired_order, fill_value=0)

# print the confusion matrix
print(confusion_matrix)

'''


: 

: 